# Find a predur for the Cone

In [ ]:
import importlib

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
import time

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '_pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils
import compartments_and_predur_utils as utils

# Cone

In [ ]:
params_default = data_utils.load_var(os.path.join('cell_params', 'cone_cell_params_default.pkl'))
params_unit = data_utils.load_var(os.path.join('cell_params', 'cone_cell_params_unit.pkl'))

In [ ]:
data_folder = os.path.join('..', 'ExperimentalData', 'PreprocessedData')
stimulus    = pd.read_csv(os.path.join(data_folder, 'ConeData_stimulus_time_and_amp_corrected.csv'))

stimulus.plot(x='Time', figsize=(15,2))
plt.show()

In [ ]:
import retsim_cells
importlib.reload(retsim_cells)

# Create cell.
cell = retsim_cells.Cone(
    predur=0.1, t_rng=(1.9,2.2),
    params_default=params_default, params_unit=params_unit,
    stimulus=stimulus, stim_type='Light',
    cone_densfile       = 'dens_cone_optimize_cone.n',
    nval_file           = 'nval_cone_optimize_cone.n',
    chanparams_file     = 'chanparams_cone_optimize_cone.n',
    expt_file_list      = ['optimize_cones'],
    expt_base_file_list = [os.path.join('retsim_files', 'expt_optimize_cones.cc')],
    retsim_path=os.path.abspath(os.path.join('..', 'NeuronC', 'models', 'retsim')) + '/'
)

In [ ]:
# Create c++ file
cell.create_retsim_expt_file(verbose=True)
# Compile c++ file
!(cd {cell.retsim_path} && make)

## Check how fast resting Vm is reached.

In [ ]:
cell.init_retsim(verbose=1)

In [ ]:
# Set stimulus to background stimulus.
cell.stim[0]['Stim'] *= 0
plt.plot(cell.stim[0]['Stim'])
plt.show()

### No predur

In [ ]:
cell.rec_type='optimize'
cell.predur = 0.0
stimdur = 10

cell.update_t_rng(t_rng=(0, 0+stimdur))
nopredur_rec_data, nopredur_rec_time, nopredur_rec_stim, tdur = utils.run_and_show_cell(cell, plot=True)
print(tdur)

In [ ]:
for col in nopredur_rec_data.columns:
    if 'Vm' in col:
        utils.plot_reaching_eq(
            nopredur_rec_data, nopredur_rec_time,
            nopredur_rec_stim, plt_Vm_name=col,
            Vm_eq=nopredur_rec_data[col].iloc[-1]
        )

### Run with small predur.

In [ ]:
cell.predur = 1.0
stimdur = 5

cell.update_t_rng(t_rng=(0, 0+stimdur))
smallpredur_rec_data, smallpredur_rec_time, smallpredur_rec_stim, tdur = utils.run_and_show_cell(cell, plot=True)
print(tdur)

In [ ]:
for col in smallpredur_rec_data.columns:
    if 'Vm' in col:
        utils.plot_reaching_eq(
            smallpredur_rec_data, smallpredur_rec_time,
            smallpredur_rec_stim, plt_Vm_name=col,
            Vm_eq=nopredur_rec_data[col].iloc[-1]
        )

### Run with different predurs

In [ ]:
predur_list = [0.1, 0.5, 1, 3, 4, 5, 6, 10]

predur_tdur_list = []
predur_rec_data_list = []
cell.update_t_rng(t_rng=(0, 0+0.01))

for idx, predur in enumerate(predur_list):
    cell.predur = predur
    rec_data, rec_time, rec_stim, tdur = utils.run_and_show_cell(cell, plot=idx==0)
    print(predur, end='\t')
    
    # Save data.
    predur_tdur_list.append(tdur)
    predur_rec_data_list.append(rec_data)

In [ ]:
importlib.reload(utils)

In [ ]:
utils.plot_params_vs_tdur(params=predur_list, tdur_list=predur_tdur_list)
print(f'{predur_tdur_list[-1] / predur_list[-1]:.2f} seconds per predur seconds')

In [ ]:
utils.plot_params_vs_time_to_Vm_eq(
    params=predur_list, rec_data_list=predur_rec_data_list,
    Vm_eq=nopredur_rec_data["Vm Soma"].iloc[-1], plt_Vm_name="Vm Soma"
)

### Run with different stim times

In [ ]:
stimdur_list = [0.1, 0.5, 1, 3, 6]

stimdur_tdur_list = []
stimdur_rec_data_list = []

cell.predur = 0.0

for idx, stimdur in enumerate(stimdur_list):
    cell.update_t_rng(t_rng=(0, stimdur))
    rec_data, rec_time, rec_stim, tdur = utils.run_and_show_cell(cell, plot=idx==0)
    
    # Save data.
    stimdur_tdur_list.append(tdur)
    stimdur_rec_data_list.append(rec_data)

In [ ]:
utils.plot_params_vs_tdur(params=stimdur_list, tdur_list=stimdur_tdur_list)
print(f'{stimdur_tdur_list[-1] / stimdur_list[-1]:.2f} seconds per sim seconds')

# Test sim_dt for optimize cpl

In [ ]:
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.05] = 0.3
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.1] = 1
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.15] = 0.5
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.2] = 0.0
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.25] = 0.5
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.3] = 0.0
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.35] = 1.0

plt.plot(cell.stim[0]['Time'], cell.stim[0]['Stim'])
plt.xlim(0, 0.4)
plt.show()

In [ ]:
cell.rec_type='optimize'
cell.predur = 6.0
cell.update_t_rng((0, 0.4))

In [ ]:
sim_dt_list = [5e-7, 1e-6, 2e-6, 5e-6, 1e-5, 2e-5, 5e-5, 1e-4]

In [ ]:
sim_dt_rec_data_list = []
sim_dt_rec_time_list = []

for sim_dt in sim_dt_list:
    print(sim_dt)
    
    cell.sim_dt = sim_dt
    
    sim_dt_rec_data, sim_dt_rec_time, _, tdur = utils.run_and_show_cell(cell, plot=False)
    assert sim_dt_rec_data is not None
    print(f'min = {tdur/60}')
    
    sim_dt_rec_data_list.append(sim_dt_rec_data)
    sim_dt_rec_time_list.append(sim_dt_rec_time)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15,6))
for rec_time, rec_data, sim_dt in zip(sim_dt_rec_time_list, sim_dt_rec_data_list, sim_dt_list):
    axs[0].plot(rec_time, rec_data['rate Cone'], label=sim_dt)
    axs[1].plot(rec_time, rec_data['Vm Soma'], label=sim_dt)
    
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15,6))
for rec_time, rec_data, sim_dt in zip(sim_dt_rec_time_list, sim_dt_rec_data_list, sim_dt_list):
    axs[0].plot(
        rec_time, rec_data['rate Cone'] - sim_dt_rec_data_list[0]['rate Cone'], label=sim_dt
    )
    axs[1].plot(
        rec_time, (rec_data['Vm Soma'] - sim_dt_rec_data_list[0]['Vm Soma'])*1e3,
        label=sim_dt
    )
axs[1].set_ylabel('mV')
plt.legend()
plt.show()